# Alimentador obra - ubicación

Notebook para completar la ubicación de obras con ciudad y país anterior

In [1]:
import os
import json
import numpy as np
import requests
import pandas as pd
from dotenv import load_dotenv

## Abrir tablas y filtrar datos



In [9]:
load_dotenv()
key = os.getenv('KEY')
config = json.load(open('../config/config.json'))

filtrar = False

if filtrar:
    obras_arriba = '../datos/salida/obra.csv'
    obras_abajo = config['csv']


    # dataframes de las tablas
    obras_a = pd.read_csv(obras_arriba, dtype=str)
    obras_b = pd.read_csv(obras_abajo, dtype=str)

    #filtrar los simbolos por los ids de obras en arca
    obras_filtradas = obras_b.loc[obras_b['t'].isin(obras_a['arca_id']).tolist()]

    #muestra para prueba
    #simbolos_filtrados = simbolos_filtrados.sample(1)
    #len(simbolos_filtrados.index)
    obras = obras_filtradas

else:
    obras = pd.read_csv(config['csv'], dtype=str)

obras.dropna(how='all')

,t,pasajes_3_id,Info_categorias,pasajes_3_id.1,Info_categorias.1,pasajes_3_id.2,Info_categorias.2,autores_id,Autores,escenario_2_id,...,Gesto 3,Fisiognómica_lista,Fisiognómica,Fisiognomica_imagen_lista,fisiognómica imagen,Cartela_filacteria_2_lista,Gesto-iconotexto,Rostro_lista,Rostro,Anotación gestual
0,2,161,Desposorios de la Virgen,NaN,NaN,NaN,NaN,53,Anónimo,16,...,No,1,De pie,3,Cuerpo entero,357,Ninguna,1,Rostro 3/4,X desposorios. El modelo es el mismo y tiene d...
1,3,162,"Virgen, Nacimiento de la",NaN,NaN,NaN,NaN,53,Anónimo,15,...,No,1,De pie,3,Cuerpo entero,357,Ninguna,1,Rostro 3/4,"X natividad de la virgen, estas ligeras. Se tr..."
2,4,132,Inmaculada,NaN,NaN,NaN,NaN,53,Anónimo,17,...,No,1,De pie,3,Cuerpo entero,357,Ninguna,1,Rostro 3/4,x Virgen inmaculada con santos diversas posici...
3,5,42,Última cena,NaN,NaN,NaN,NaN,53,Anónimo,15,...,No,3,Sentado,3,Cuerpo entero,357,Ninguna,3,Frente,"X ultima cena, bajo un mismo formato casi toda..."
4,6,42,Última cena,NaN,NaN,NaN,NaN,53,Anónimo,15,...,No,3,Sentado,3,Cuerpo entero,357,Ninguna,3,Frente,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,24996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24995,24997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,24998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,24999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## URLS

In [10]:
#baseurl = 'http://localhost:8055/'
baseurl = config['baseurl']
#obra_simbolos_baseurl = f'{baseurl}items/obra_simbolos_lista'
obras_url = f'{baseurl}items/obra'
#simbolos_url = f'{baseurl}items/simbolos_lista'

headers =  {'Authorization':f'Bearer {key}'}


## Iteración

In [ ]:
for index, row in obras.iterrows():
    obra_arcaID = row['artwork_id']

    ciudad_actual = row['Ciudad Actual']
    ciudad_anterior = row['Ciudades']
    #pais_actual
    #pais_anterior
    #simbolo_arcaID = row['artwork_symbol_id']
    obra_url = f'{obras_url}?filter[arca_id][_eq]={obra_arcaID}&limit=1'
    #sim_url = f'{simbolos_url}?filter[arca_id][_eq]={simbolo_arcaID}&limit=1'
    #obra = {}
    #simbolo = {}

    #obtener la obra de la api
    try:
        r_obra = requests.get(obra_url)
        r_obra.raise_for_status()
        print(f'econtrada obra: {obra_arcaID}')
    except requests.RequestException as err:
        print(f'Error obteniendo obra: {obra_arcaID}')
        print(err)
        # si se produce error, continua el siguiente registro
        continue

    #se obtienen los datos de la respuesta
    obra = r_obra.json()['data'][0]
    print(obra)
    #obtener el simbolo de la api


    #se obtienen los datos de la respuesta
    simbolo = r_sim.json()['data'][0]
    print(simbolo)

    # crear el objeto obra simbolo con los ids de los objetos viculados
    obra_sim = {}
    obra_sim['obra_id'] = obra['id']
    obra_sim['simbolos_lista_id'] = simbolo['id']
    
    # crear m2m obra simbolo
    try:
        r_obsim_m2m = requests.post(obra_simbolos_baseurl, json = obra_sim, headers=headers)
        r_obsim_m2m.raise_for_status()
    except requests.RequestException as err:
        print(f'Error creando m2m obra simbolo: {obra_sim}')
        print(err)
        continue

    obsim_m2m = r_obsim_m2m.json()['data']
    print('Creada relacion obra simbolo')
    print(obsim_m2m)
    